In [1]:
from __future__ import print_function
from pathlib import Path
from random import shuffle
import random

import argparse
import copy
import numpy as np
import sys
import tensorflow as tf
import scipy.sparse as sp
import sys

import sys
sys.path.append("..") 
from deeplp.models.deeplp_att import DeepLP_ATT
from deeplp.models.deeplp_edge import DeepLP_Edge
from deeplp.models.deeplp_wrbf import DeepLP_WRBF
from deeplp.models.lp import LP
from deeplp.utils import (calc_masks, create_seed_features, load_data,
                          num_layers_dict, prepare_data, random_unlabel)

/Users/kojin/anaconda/envs/ml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
true_labels, edge_features, graph \
= load_data('linqs_cora',model='edge')

labeled_indices, unlabeled_indices = \
random_unlabel(true_labels,0.01,
   seed=1)
num_nodes, num_classes = true_labels.shape

labels, is_labeled = calc_masks(true_labels, labeled_indices, unlabeled_indices)

In [31]:
from scipy.sparse.linalg import eigs as eigs
from scipy.sparse.csgraph import laplacian as lap
from sklearn.preprocessing import normalize

In [47]:
l = lap(graph)

In [48]:
l.shape

(2485, 2485)

In [51]:
eigs(m,k=2,which='LR')

(array([ 3848.38445552+0.j,  3109.73393534+0.j]),
 array([[ -4.64110389e-20+0.j,  -2.02578057e-19+0.j],
        [  3.81685969e-19+0.j,   1.31394082e-18+0.j],
        [ -3.53783280e-19+0.j,  -7.87159040e-19+0.j],
        ..., 
        [  2.54094511e-19+0.j,   1.00516592e-18+0.j],
        [  4.16253206e-19+0.j,   1.12867624e-18+0.j],
        [  2.83508208e-19+0.j,   6.36090046e-19+0.j]]))

In [53]:
3109.73393534/3848.38445552, 105/169.01414966

(0.8080621807105308, 0.6212497605154653)

In [6]:
from deeplp.utils import (sigma_for_rbf,rbf,load_graph,edge_np_to_csr)

In [7]:
U, D, B, R, node_features = load_graph('linqs_cora')


In [8]:
edges = np.array(B.edges())
sources, sinks = edges[:, 0], edges[:, 1]

In [9]:
p='../data/linqs_cora/features_raw.csv'
x=np.loadtxt(p, delimiter=',')

In [10]:
s=sigma_for_rbf(x)

In [11]:
from sklearn.metrics.pairwise import (euclidean_distances, paired_distances,
                                      rbf_kernel,)

In [12]:
row_features = x[sources]
col_features = x[sinks]
d=paired_distances(row_features, col_features, metric=rbf)

In [13]:
n=np.column_stack((sources,sinks))

In [17]:
from scipy.sparse import coo_matrix, csr_matrix

In [15]:

def edge_np_to_csr(graph, vals=[]):
    """
    Convert np array with each row being (row_index,col_index,value)
    of a graph to a scipy csr matrix.
    """
    num_nodes = int(max(max(graph[:, 0]), max(graph[:, 1])) + 1)
    if len(vals) == 0:
        vals = np.ones(len(graph[:, 0]))
    csr = csr_matrix(
        (vals, (graph[:, 0], graph[:, 1])), shape=(num_nodes, num_nodes))
    return csr

In [19]:
m=edge_np_to_csr(n,vals=d)

In [29]:
def _tnorm(weights):
    """Column normalize weights"""
    # row normalize T
    Tnorm = weights / np.sum(weights, axis=1, keepdims=True)
    return Tnorm

In [28]:
np.mean(graph.toarray(),axis=1)

array([ 0.00201207,  0.00160966,  0.00040241, ...,  0.00241449,
        0.00201207,  0.00120724])